In [ ]:
!pip install pandas pyarrow torch tqdm joblib

In [1]:
import pandas as pd

# ================================
# Carregar dados
# ================================
df = pd.read_parquet("1.snappy.parquet")

# Filtrar para 2022
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')
df_2022 = df[df['transaction_date'].dt.year == 2022].copy()
df_2022['semana'] = df_2022['transaction_date'].dt.to_period('W').apply(lambda r: r.start_time)

# ================================
# Remover outliers IQR=1.5
# ================================
def remove_outliers(df, col="quantity", k=1.5):
    def _iqr_filter(x):
        q1 = x.quantile(0.25)
        q3 = x.quantile(0.75)
        iqr = q3 - q1
        lower = q1 - k * iqr
        upper = q3 + k * iqr
        return x[(x >= lower) & (x <= upper)]
    return df.groupby(["internal_product_id", "internal_store_id"], group_keys=False).apply(
        lambda g: g.assign(**{col: _iqr_filter(g[col])})
    )

df_clean = remove_outliers(df_2022, col="quantity", k=1.5)

# ================================
# Média móvel dos 2 últimos valores
# ================================
df_clean = df_clean.sort_values(["internal_product_id", "internal_store_id", "semana"])
df_clean["quantity_smooth"] = df_clean.groupby(["internal_product_id", "internal_store_id"])['quantity'] \
    .transform(lambda x: x.rolling(2, min_periods=1).mean())

# ================================
# Contagem antes e depois da remoção de duplicados
# ================================
# Antes
semanas_por_prod_pdv_before = df_clean.groupby(['internal_product_id', 'internal_store_id'])['semana'].nunique()
count_before = (semanas_por_prod_pdv_before >= 6).sum()

# Remover duplicados
df_clean = df_clean.drop_duplicates(subset=["internal_product_id", "internal_store_id", "semana"])

# Depois
semanas_por_prod_pdv_after = df_clean.groupby(['internal_product_id', 'internal_store_id'])['semana'].nunique()
count_after = (semanas_por_prod_pdv_after >= 12).sum()

print(f"🔹 Produtos/PDVs com >= 12 semanas (antes): {count_before}")
print(f"🔹 Produtos/PDVs com >= 12 semanas (depois): {count_after}")

# ================================
# Média de semanas presentes por produto/pdv
# ================================
media_semanas_prod_pdv = semanas_por_prod_pdv_after.mean()
print(f"📊 Média de semanas presentes por produto/pdv após limpeza: {media_semanas_prod_pdv:.2f}")

# ================================
# Salvar dados tratados
# ================================
df_clean.to_parquet("dados_tratados.parquet", index=False)
print("✅ Arquivo 'dados_tratados.parquet' gerado para download.")



KeyboardInterrupt



In [ ]:
import pandas as pd

# Carregar dados tratados
df = pd.read_parquet("dados_tratados.parquet")

# Remover linhas com NaN na coluna 'quantity'
df_filtrado = df.dropna(subset=['quantity', 'quantity_smooth'])

# Salvar novo arquivo sem NaN
df_filtrado.to_parquet("dados_tratados_sem_nan.parquet", index=False)
print("Arquivo 'dados_tratados_sem_nan.parquet' gerado sem valores NaN em 'quantity'.")



In [2]:
# ================================
# !pip install pandas pyarrow torch tqdm joblib
# ================================

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
import glob
import os
import time
import joblib
import warnings

warnings.filterwarnings("ignore")

# ================================
# Configurações principais
# ================================
BATCH_SIZE = 50000
OUTPUT_PREFIX = "previsoes_2023_mlp_batch"
N_SEMANAS_PREV = 5
WINDOW_DEFAULT = 10  # tamanho da janela padrão da MLP
MIN_SEMANAS_MLP = 10  # mínimo de semanas para usar MLP

# ================================
# Configuração GPU
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🚀 Usando dispositivo:", device)

# ================================
# Função para carregar dados
# ================================
def carregar_dados():
    df = pd.read_parquet("dados_tratados_sem_nan.parquet")
    df = df.rename(columns={
        "internal_store_id": "pdv",
        "internal_product_id": "produto",
        "quantity": "quantidade"
    })
    df = df.sort_values(["produto", "pdv", "semana"])
    df["produto"] = df["produto"].astype(str).str.strip()
    df["pdv"] = df["pdv"].astype(str).str.strip()
    return df[["semana", "pdv", "produto", "quantidade", "quantity_smooth"]]

# ================================
# Rede Neural Global (MLP simples)
# ================================
class MLP(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

# ================================
# Função de previsão MLP
# ================================
def prever_mlp(model, serie, window=WINDOW_DEFAULT, n_semanas=N_SEMANAS_PREV):
    ultimos = serie[-window:] if len(serie) >= window else [0]*(window - len(serie)) + serie
    preds = []
    model.eval()
    with torch.no_grad():
        for _ in range(n_semanas):
            inp = torch.tensor(ultimos[-window:], dtype=torch.float32, device=device).unsqueeze(0)
            pred = model(inp).item()
            preds.append(int(round(pred)))
            ultimos.append(pred)
    return preds

# ================================
# Wrapper de previsão por produto/pdv
# ================================
def prever_grupo(produto, pdv, grupo, model):
    serie = grupo["quantity_smooth"].fillna(method="ffill").dropna().tolist()
    if len(serie) < MIN_SEMANAS_MLP:
        return []  # Ignora séries curtas

    previsoes = prever_mlp(model, serie)
    return [
        {"semana": i+1, "pdv": pdv, "produto": produto, "quantidade": q}
        for i, q in enumerate(previsoes)
    ]

# ================================
# Execução principal
# ================================
if __name__ == "__main__":
    df = carregar_dados()

    # Carregar modelo MLP treinado
    window = joblib.load("mlp_global_metadata.pkl")["window"]
    model = MLP(window).to(device)
    model.load_state_dict(torch.load("mlp_global_2022.pth"))
    model.eval()

    # Remover batches antigos
    for f in glob.glob(f"{OUTPUT_PREFIX}_*.parquet"):
        os.remove(f)

    resultados, batch_idx = [], 0
    start_time = time.time()

    grupos_iter = df.groupby(["produto", "pdv"])
    total = len(grupos_iter)

    # Loop sequencial sem multiprocessamento
    for idx, ((produto, pdv), grupo) in enumerate(grupos_iter, start=1):
        try:
            previsoes = prever_grupo(produto, pdv, grupo, model)
            resultados.extend(previsoes)
        except Exception as e:
            print(f"Erro em {produto}, {pdv}: {e}")

        # Salvar batch
        if len(resultados) >= BATCH_SIZE:
            batch_idx += 1
            batch_file = f"{OUTPUT_PREFIX}_{batch_idx}.parquet"
            pd.DataFrame(resultados).to_parquet(batch_file, index=False, engine="pyarrow")
            resultados = []

        # ETA simples
        if idx % 1000 == 0 or idx == total:
            elapsed = time.time() - start_time
            rate = idx / elapsed if elapsed > 0 else 0.1
            eta = (total - idx) / rate if rate > 0 else 0
            print(f"{idx}/{total} combos processados, ETA ≈ {int(eta//3600)}h {int((eta%3600)//60)}m {int(eta%60)}s")

    # Salvar últimos resultados
    if resultados:
        batch_idx += 1
        batch_file = f"{OUTPUT_PREFIX}_{batch_idx}.parquet"
        pd.DataFrame(resultados).to_parquet(batch_file, index=False, engine="pyarrow")

    # Concatenar todos os batches
    all_files = sorted(glob.glob(f"{OUTPUT_PREFIX}_*.parquet"))
    df_final = pd.concat([pd.read_parquet(f) for f in all_files], ignore_index=True)
    df_final.to_parquet("previsoes_2023_mlp.parquet", index=False, engine="pyarrow")

    print("✅ Arquivo 'previsoes_2023_mlp.parquet' gerado com sucesso!")
    print(df_final.head(20))


🚀 Usando dispositivo: cuda
1000/1044310 combos processados, ETA ≈ 6h 37m 0s
2000/1044310 combos processados, ETA ≈ 3h 30m 23s
3000/1044310 combos processados, ETA ≈ 2h 25m 1s
4000/1044310 combos processados, ETA ≈ 1h 52m 33s
5000/1044310 combos processados, ETA ≈ 1h 36m 7s
6000/1044310 combos processados, ETA ≈ 1h 25m 34s
7000/1044310 combos processados, ETA ≈ 1h 17m 30s
8000/1044310 combos processados, ETA ≈ 1h 11m 46s
9000/1044310 combos processados, ETA ≈ 1h 6m 52s
10000/1044310 combos processados, ETA ≈ 1h 1m 3s
11000/1044310 combos processados, ETA ≈ 0h 55m 55s
12000/1044310 combos processados, ETA ≈ 0h 52m 38s
13000/1044310 combos processados, ETA ≈ 0h 49m 44s
14000/1044310 combos processados, ETA ≈ 0h 46m 58s
15000/1044310 combos processados, ETA ≈ 0h 45m 24s
16000/1044310 combos processados, ETA ≈ 0h 42m 49s
17000/1044310 combos processados, ETA ≈ 0h 40m 31s
18000/1044310 combos processados, ETA ≈ 0h 38m 28s
19000/1044310 combos processados, ETA ≈ 0h 36m 59s
20000/1044310 combo

In [ ]:
# ================================
# !pip install pandas pyarrow torch tqdm joblib
# ================================

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import warnings
import joblib
import time

warnings.filterwarnings("ignore")

# ================================
# Configuração GPU
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🚀 Usando dispositivo:", device)

# ================================
# Função para carregar dados
# ================================
def carregar_dados():
    df = pd.read_parquet("dados_tratados_sem_nan.parquet")
    df = df.rename(columns={
        "internal_store_id": "pdv",
        "internal_product_id": "produto",
        "quantity": "quantidade"
    })
    df = df.sort_values(["produto", "pdv", "semana"])
    df["produto"] = df["produto"].astype(str).str.strip()
    df["pdv"] = df["pdv"].astype(str).str.strip()
    return df[["semana", "pdv", "produto", "quantidade", "quantity_smooth"]]

# ================================
# Rede Neural Global (MLP simples)
# ================================
class MLP(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

# ================================
# Preparar dataset global
# ================================
def preparar_dataset_global(df, window=10):
    X, y, grupos = [], [], []
    for (produto, pdv), grupo in df.groupby(["produto", "pdv"]):
        serie = grupo["quantity_smooth"].fillna(method="ffill").dropna().values
        if len(serie) < 10:  # apenas séries com ≥10 semanas
            continue
        for i in range(len(serie) - window):
            X.append(serie[i:i+window])
            y.append(serie[i+window])
            grupos.append((produto, pdv))
    return np.array(X), np.array(y), grupos

# ================================
# Execução principal
# ================================
if __name__ == "__main__":
    # Parâmetros
    window = 10
    batch_size = 1024
    num_epochs = 30

    # Carregar dados
    df = carregar_dados()
    print("📊 Preparando dataset global...")
    X, y, grupos = preparar_dataset_global(df, window=window)

    print(f"🔹 Séries válidas: {len(grupos)}, amostras: {len(X)}")

    # Tensores
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

    dataset = TensorDataset(X_tensor, y_tensor)
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        pin_memory=True,
        num_workers=0  # remove multiprocessamento
    )

    # Modelo, loss e otimizador
    model = MLP(window).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Treino
    start_time = time.time()
    print("⚡ Treinando modelo global...")

    for epoch in range(1, num_epochs+1):
        epoch_start = time.time()
        running_loss = 0.0
        for xb, yb in loader:
            xb = xb.to(device)
            yb = yb.to(device)
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_loss = running_loss / len(loader)
        epoch_time = time.time() - epoch_start
        print(f"✅ Epoch {epoch}/{num_epochs} finalizado em {int(epoch_time//60)}m {int(epoch_time%60)}s, "
              f"Loss médio: {avg_loss:.4f}")

    total_time = time.time() - start_time
    print(f"\n⏱️ Treinamento completo em {int(total_time//3600)}h {int((total_time%3600)//60)}m {int(total_time%60)}s")

    # Salvar modelo
    torch.save(model.state_dict(), "mlp_global_2022.pth")
    joblib.dump({"window": window, "grupos": grupos}, "mlp_global_metadata.pkl")
    print("✅ Modelo global treinado e salvo!")
